In [5]:
from ortools.constraint_solver import routing_enums_pb2, pywraplp
from folium import Map, Marker, PolyLine

# Sample delivery locations (replace with your data)
locations = [
    {'id': 1, 'lat': 37.7749, 'lon': -122.4194},  # San Francisco
    {'id': 2, 'lat': 34.0522, 'lon': -118.2437},  # Los Angeles
    {'id': 3, 'lat': 33.4483, 'lon': -112.0776},  # Phoenix
    {'id': 4, 'lat': 47.6062, 'lon': -122.3321},  # Seattle
    {'id': 5, 'lat': 39.7392, 'lon': -104.9903},  # Denver
]

# Define distance function (replace with actual calculation using OSM distance matrix API or coordinates)
def distance(loc1, loc2):
    # Replace with your distance calculation logic (e.g., using OSM distance matrix API)
    return abs(loc1['lat'] - loc2['lat']) + abs(loc1['lon'] - loc2['lon'])

# Create distance matrix from locations
distance_matrix = [[distance(loc1, loc2) for loc2 in locations] for loc1 in locations]

# Create routing model
manager = pywraplp.Solver.CreateSolver('CBC')
routing = routing_enums_pb2.RoutingModel(manager)

# Define vehicle and depot
depot = 0
n_vehicles = 1
vehicle_capacities = [10]  # Adjust capacity as needed

# Define distance callback
def distance_callback(from_index, to_index):
    return distance_matrix[from_index][to_index]

routing.SetDistanceCallback(distance_callback)

# Create dimension for vehicle capacity
transit_callback = lambda from_index, to_index: 0  # No transit cost between locations
dimension_name = 'Capacity'
routing.AddDimensionWithVehicleCapacity(transit_callback, 0, vehicle_capacities, dimension_name, True)
routing.SetFixedCostCallback(lambda from_index, to_index: 0)  # No fixed cost

# Search parameters
search_parameters = pywraplp.DefaultRoutingSearchParameters()
search_parameters.first_solution_strategy = routing_enums_pb2.FirstSolutionStrategy.GLOBAL_CHEAPEST_ARC

# Solve the VRP
assignment = routing.SolveWithParameters(search_parameters)

# Check solution status
if assignment.status() != routing_enums_pb2.RoutingModel.OPTIMAL:
    print("Solution not optimal!")
    exit()

# Extract optimized route
route = assignment.solution().GetValues(routing.NextVar())
optimized_route = [locations[i] for i in route]

# Create initial Leaflet map centered on the first location
m = Map(location=[optimized_route[0]['lat'], optimized_route[0]['lon']], zoom_start=6)

# Add markers for each location
for loc in optimized_route:
    Marker([loc['lat'], loc['lon']], popup=f"Stop {loc['id']}").add_to(m)

# Create PolyLine for the route
route_coords = [(loc['lat'], loc['lon']) for loc in optimized_route]
folium.PolyLine(route_coords, color='blue', weight=2).add_to(m)

# Display the optimized route
m


ImportError: cannot import name 'pywraplp' from 'ortools.constraint_solver' (/opt/homebrew/lib/python3.11/site-packages/ortools/constraint_solver/__init__.py)